In [96]:
import pandas as pd
import joblib
from sklearn.metrics import r2_score
import numpy as np
import requests

In [90]:
#import trained models
with open('LinReg VII.pkl', 'rb') as f:
    linreg = joblib.load(f)
with open('Lasso VII.pkl', 'rb') as f:
    lasso = joblib.load(f)
with open('Ridge VII.pkl', 'rb') as f:
    ridge = joblib.load(f)

In [3]:
df = pd.read_csv('gs://vertexaioutputs/Historical Financial Data With ER and AR Calulated Updated.csv',nrows=10000000)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
prediction = pd.read_csv('gs://vertexaioutputs/Training Set Two Million Updated.csv')

In [5]:
print(prediction)

               Time Ticker          Sector  BETA_SEC1M   BETA_MRKT1M  \
0        2007-01-03      A      Healthcare   -0.072605 -8.837373e-03   
1        2007-01-03    ABC      Healthcare   -0.057317 -6.976599e-03   
2        2007-01-03   ABMD      Healthcare   -0.068717 -8.364233e-03   
3        2007-01-03    ABT      Healthcare   -0.066021 -8.036085e-03   
4        2007-01-03   ACAD      Healthcare   -0.085997 -1.046746e-02   
...             ...    ...             ...         ...           ...   
1999995  2009-07-20  BACpB  Sector Unknown    0.000354  9.697865e-07   
1999996  2009-07-20  BACpE  Sector Unknown   -0.080559 -2.208046e-04   
1999997  2009-07-20  BACpL  Sector Unknown   -0.025404 -6.962871e-05   
1999998  2009-07-20    BAL  Sector Unknown   -0.114941 -3.150414e-04   
1999999  2009-07-20  BANFP  Sector Unknown   -0.040680 -1.114999e-04   

         BETA_SEC3M  BETA_MRKT3M  BETA_SEC6M  BETA_MRKT6M  BETA_SEC1Y  ...  \
0         -0.068261    -0.008309   -0.065944    -0.008027

In [190]:
def score_model(Y, hat, prediction_sample, prediction, t):
    #score linreg model by calculating percent difference from actual return
    scores = pd.DataFrame(abs((Y-hat)/Y))
    scores['Ticker'] = prediction[['Ticker']]
    if t is '1M':
        scores = scores.rename(columns={'AR_1M':'error','Ticker':'ticker'})
    else:
        scores = scores.rename(columns={'AR_3M':'error','Ticker':'ticker'})
    #print(scores_linreg)
    print(scores.sort_values(by='error').head(20))
    
    print('R2')
    print(r2_score(Y,hat))
    
    print('Model accurate within 10%')
    from collections import Counter
    acc = Counter(scores['error'] < 0.1)
    print(acc[True]/len(scores))
    
    return scores

In [142]:
def setup_model(df_sample, prediction_sample, hat, scores):
    #get tickers, volumes, and marketcaps
    ticker_info = df_sample
    tickers = ticker_info.loc[:,'Ticker']
    volumes = ticker_info.loc[:]['v']
    marketcaps = ticker_info.loc[:]['v'] * ticker_info.loc[:]['c']

    #linear regression
    #get tickers that are in both datasets
    sample_sectors = prediction_sample[prediction_sample['Ticker'].isin(list(tickers))][:]

    #get linear regression results from smaller sample
    sample_hat = pd.DataFrame(hat[list(sample_sectors.reset_index().index)])

    #get scores for smaller sample
    sample_scores = scores[scores['ticker'].isin(list(tickers))][:]

    #prep data for allocation model
    beta = sample_sectors.loc[:,'BETA_MRKT1M'].tolist()
    accuracy = 1 - sample_scores['error']*sample_scores['error']
    ticker = sample_sectors.loc[:,'Ticker'].tolist()

    #set up data for allocation model
    d = {
        'beta':beta,
        'volume':volumes.tolist(),
        'marketcap':marketcaps.tolist(),
        'expected_return':sample_hat[0].tolist(),
        'accuracy':accuracy.tolist(),
        'ticker':ticker
    }

    stocks = pd.DataFrame(data=d)
    stocks.set_index('ticker',inplace=True)
    return stocks


In [143]:
def priority_model(stocks):
    beta_weight = -0.2
    volume_weight = 0.25
    marketcap_weight = 0.15
    return_weight = 0.25

    priority = list()
    beta_contributions = list()
    volume_contributions = list()
    marketcap_contributions = list()
    return_contributions = list()


    for ind, stock in stocks.iterrows():
        beta_contribution = abs(stock.beta)*beta_weight
        beta_contributions.append(beta_contribution)
        if (stock.volume > 0.0):
            volume_contribution = np.math.log(stock.volume) * volume_weight
        else:
            volume_contribution = 0.0
        volume_contributions.append(volume_contribution)
        if (stock.marketcap > 0.0):
            marketcap_contribution = np.math.log(stock.marketcap) * marketcap_weight
        else:
            marketcap_contribution = 0.0
        marketcap_contributions.append(marketcap_contribution)
        return_contribution = np.sign(stock.expected_return)*np.math.log(abs(stock.expected_return*100))* return_weight
        return_contributions.append(return_contribution)
        if stock.accuracy > 0.0:
            stock_priority = stock.accuracy * (beta_contribution + volume_contribution + marketcap_contribution + return_contribution )
        else:
            stock_priority = 0.0
        priority.append(stock_priority)



    stocks['beta_contribution'] = beta_contributions
    stocks['volume_contribution'] = volume_contributions
    stocks['marketcap_contribution'] = marketcap_contributions
    stocks['return_contribution'] = return_contributions
    stocks['priority'] = priority

    stocks.sort_values(by='priority',ascending=False,inplace=True)
    top_priorities = stocks[0:10]
    print(top_priorities)
    normalized_values = dict()
    for i,row in top_priorities.iterrows():
        nv = row['priority'] / sum(top_priorities['priority'])
        normalized_values[i] = nv

    print(normalized_values)

    portfolio_beta = 0
    for i in normalized_values.keys():
        portfolio_beta = portfolio_beta + normalized_values[i]*stocks.loc[i]['beta']

    print(portfolio_beta)
    
    return normalized_values

In [145]:
def print_return(normalized_values, date_start, date_end):
    portfolio_return = 0.0
    for i in normalized_values.keys():
        print(i)
        temp = df.loc[df['Ticker'] == i]
        start_date = temp.loc[temp.loc[:]['Time'] == date_start].iloc[0]
        end_date = temp.loc[temp.loc[:]['Time'] >= date_end].iloc[0]
        ret_val = (end_date['c'] - start_date['o'])/start_date['o']
        print(ret_val)
        portfolio_return += ret_val*normalized_values[i]

    print('Overall return')
    print(portfolio_return)

In [120]:
def get_spy_return(start_date, end_date):
    spy_start = requests.get(f"https://api.polygon.io/v1/open-close/SPY/" + start_date + "?adjusted=true&apiKey=el0FbXEICAXLDOh_2HHODn_QdqmBj_5WlhdNvp").json()
    spy_end   = requests.get(f"https://api.polygon.io/v1/open-close/SPY/" + end_date   + "?adjusted=true&apiKey=el0FbXEICAXLDOh_2HHODn_QdqmBj_5WlhdNvp").json()
    
    
    print('SPY Return ' + start_date + ' to ' + end_date)
    print((spy_end['close'] - spy_start['open'])/spy_start['open'])
    

In [173]:
def allocation_model(full_data, prediction, date_start, date_end, model,t):
    df_sample = full_data.loc[full_data.loc[:,'Time'] == date_start]
    prediction_sample = prediction.loc[prediction.loc[:,'Time'] == date_start]
    if t is '1M':
        X = prediction_sample[['BETA_SEC1M', 'BETA_MRKT1M', 'ER_MRKT1M', 'ER_SEC1M', 'RFR1M']]
        Y = prediction_sample[['AR_1M']]
    elif t is '3M':
        X = prediction_sample[['BETA_SEC3M', 'BETA_MRKT3M', 'ER_MRKT3M', 'ER_SEC3M', 'RFR3M']]
        Y = prediction_sample[['AR_3M']]
    else:
        print('Bad model')
        return
    
    hat = model.predict(X)
    hat = hat.reshape(len(hat),1)
    
    scores = score_model(Y, hat, prediction_sample, prediction, t)
    stocks = setup_model(df_sample, prediction_sample, hat, scores)
    norm_vals = priority_model(stocks)
    print_return(norm_vals, date_start, date_end)
    get_spy_return(date_start, date_end)

In [191]:
date_start = '2009-03-20'
date_end = '2009-04-20'

allocation_model(df, prediction,date_start,date_end, linreg,'1M')
allocation_model(df, prediction,date_start,date_end, lasso,'1M')
allocation_model(df, prediction,date_start,date_end, ridge,'1M')

            error ticker
1729069  0.001217    BTZ
1729808  0.001575   FAST
1730065  0.002466   AKAM
1729695  0.002477     WU
1728581  0.005131   AEZS
1729890  0.005737    LII
1728830  0.006030    WAT
1729618  0.006352   TCBI
1731198  0.010117   CBON
1729774  0.010472    CSX
1731653  0.012128    RZV
1729938  0.012790   ODFL
1730187  0.012870    HPQ
1728900  0.013116    JHX
1729160  0.015156   EBTC
1730282  0.015553   RELL
1729281  0.018031    GRF
1728918  0.018644    NAK
1729766  0.018657     CP
1730592  0.018662   CTRN
R2
0.0030103258299331026
Model accurate within 10%
0.04905308910276312
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
AAPL    0.000034  6.957017e+08  2.524145e+09         0.283654  0.997018   
JPM     0.000152  1.089646e+08  2.522529e+09         0.468712  0.979961   
BAC     0.000214  5.740042e+08  3.553086e+09         0.468712  0.905408   
HPQ     0.000028  6.223501e+0

In [192]:
date_start = '2009-03-19' 
date_end = '2009-06-19'
with open('LinReg VII 3M.pkl', 'rb') as f:
    linreg3 = joblib.load(f)
with open('Lasso VII 3M.pkl', 'rb') as f:
    lasso3 = joblib.load(f)
with open('Ridge VII 3M.pkl', 'rb') as f:
    ridge3 = joblib.load(f)
    
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1726982  0.000298   KVHI
1725651  0.000602    FRD
1727383  0.002110    FLL
1727378  0.002110    EVK
1727677  0.003611    AKR
1725760  0.003878    AGM
1726500  0.004042    AZZ
1727018  0.004535   NTAP
1725494  0.004556   LMAT
1727058  0.004557   SCSC
1726740  0.004795   RFIL
1726095  0.005464    IFN
1727479  0.006465     OI
1726107  0.007167     IX
1726752  0.007441    SCS
1727156  0.007583    EVC
1726517  0.008375    CAT
1727634  0.008410   NAII
1727470  0.009109    MYE
1726799  0.011461    UUU
R2
0.002817474551902177
Model accurate within 10%
0.0411509900990099
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
BAC    -0.000003  6.234936e+08  4.320811e+09         0.988978  0.995210   
AAPL   -0.000005  5.003179e+08  1.815804e+09         0.502184  0.984159   
AXP     0.000347  3.334679e+07  4.358425e+08         0.988988  0.986716   
HPQ    -0.000003  4.228331e+07 

In [193]:
date_start = '2008-09-02'
date_end = '2008-10-02'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')


            error ticker
1286293  0.000343   TZOO
1286472  0.000485   GIII
1285609  0.000702     CP
1285821  0.001196  RYAAY
1285208  0.001553    LNC
1284851  0.002250    AWP
1285630  0.003374    DHX
1285580  0.004763   BBSI
1284839  0.005417   ARCC
1284849  0.006083    AVK
1285725  0.006141    KBR
1286484  0.007607    HAS
1285978  0.007953    DBX
1285799  0.008354   PLPC
1286024  0.009370   GSIT
1285270  0.009906    MXF
1285762  0.011196    MWA
1285853  0.011376    TKR
1286386  0.011386    AZO
1287010  0.011780  BACpB
R2
0.003680144511652217
Model accurate within 10%
0.05564238824632698
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
MSFT    0.076558  6.608226e+07  1.790829e+09        -0.114218  0.998991   
HPQ     0.076977  4.394599e+07  9.177417e+08        -0.114218  0.997512   
TSM     0.076435  2.524243e+07  2.350928e+08        -0.114218  0.996569   
GE      0.083529  7.073126e+06

In [194]:
date_start = '2008-09-02'
date_end = '2008-12-02'
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1287290  0.000486    IYH
1286568  0.000592   RENT
1284891  0.000806    BME
1286696  0.001051   LANC
1285512  0.001227    VPV
1285892  0.001365    ZNH
1285607  0.001866    CNI
1284982  0.003019   DCOM
1284883  0.003400    BKN
1285150  0.005356   HTGC
1287043  0.005824    COW
1287454  0.005903    RHS
1284886  0.005907    BLE
1285679  0.005937    GPN
1285820  0.007189   RVSN
1285289  0.007568   NECB
1285561  0.007653    AOS
1287383  0.007979    PGF
1285045  0.008720    EVV
1285204  0.008746    LEO
R2
-0.010217828597986811
Model accurate within 10%
0.04751484839012191
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
WFC     0.068434  3.768553e+07  1.176165e+09        -0.214515  0.989398   
HPQ     0.076977  4.394599e+07  9.177417e+08        -0.150206  0.964629   
VRTX    0.070258  1.789463e+06  4.783235e+07         0.148331  0.939080   
ORCL    0.079567  4.367040e+0

In [195]:
date_start = '2009-01-02'
date_end = '2009-04-02'
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1558126  0.002894   TNDM
1559007  0.006192   AAON
1558899  0.006233   SCHW
1560310  0.010030    SPG
1559645  0.010569    TXN
1559426  0.011407   CSCO
1558486  0.013631    EMO
1560710  0.014993  HOVNP
1558588  0.017919    GLV
1558585  0.017987    GLQ
1559923  0.018904      F
1558913  0.023500   SMBC
1558718  0.030802   MORN
1559304  0.032007    STN
1558181  0.032155    ESI
1558988  0.032431    WEA
1559143  0.034022    GPN
1558724  0.034046    MSD
1560526  0.037245   CRYP
1558599  0.038617   HARL
R2
0.001851273730706593
Model accurate within 10%
0.015226848974518334
            beta      volume     marketcap  expected_return  accuracy  \
ticker                                                                  
CSCO    0.000468  40883122.0  6.933777e+08         0.063536  0.999870   
BIDU    0.000460  16570630.0  2.243829e+08         0.847924  0.959541   
TXN     0.000146  15196103.0  2.437455e+08         0.063536  0.999888   
QCOM    0.000328  14757157.0  5.467527e

Stephans Stuff

In [196]:
date_start = '2009-02-02'
date_end = '2009-05-04'
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1623457  0.001450   PFIN
1623209  0.001931    ACM
1623283  0.001969    CVR
1623295  0.003470    DOV
1623108  0.003869   SIVB
1623253  0.005762    BXC
1623616  0.006080   CLFD
1622843  0.006408    IVZ
1622130  0.009032   BOLT
1622236  0.009094   MDRX
1623265  0.009277    CIR
1624543  0.009538   DMLP
1623476  0.012492   RCII
1623116  0.012940   SSBI
1622604  0.013363    CHN
1622360  0.013738    CCF
1622782  0.013826   GLRE
1623517  0.015214    TKR
1623107  0.015710    SII
1623405  0.015786    MAS
R2
0.004864664192400947
Model accurate within 10%
0.0390015600624025
            beta      volume     marketcap  expected_return  accuracy  \
ticker                                                                  
MSFT   -0.000226  88882500.0  1.584775e+09         0.297709  0.996202   
CSCO    0.000035  55291653.0  8.465152e+08         0.297730  0.997659   
BIDU    0.000877  20350330.0  2.554373e+08         1.530486  0.990652   
MS      0.000257  31275223.0  6.505246e+0

In [197]:
date_start = '2009-03-02'
date_end = '2009-06-02'
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1684711  0.001576    TKR
1684516  0.003243    FSS
1684465  0.003488   CPLP
1684997  0.003648   SMIT
1684590  0.004136    LNN
1684825  0.004295   CTSH
1685850  0.004670    WES
1685015  0.004885   SWIR
1684798  0.006416   BNSO
1683280  0.006585   AMRN
1683642  0.007993    SXT
1685834  0.008005    TGS
1684818  0.009039   CRNT
1686412  0.009135    TSL
1684731  0.009151    UUU
1684772  0.011246  AMSWA
1684673  0.011385    RHI
1684633  0.012759   NWPX
1685734  0.012967    COP
1684501  0.014593   ESLT
R2
0.0036539068971919164
Model accurate within 10%
0.036077705827937095
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
AAPL    0.000051  7.709719e+08  2.421391e+09         0.463477  0.974958   
WFC     0.000136  1.616336e+08  1.752108e+09         1.024810  0.988551   
C       0.000549  1.078407e+08  1.294088e+09         1.024812  0.969498   
MSFT    0.000021  8.059710e+

In [198]:
date_start = '2009-04-01'
date_end = '2009-07-02'
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1756176  0.000524   WOLF
1755957  0.000626   DELL
1757100  0.000754   ENTR
1754805  0.001502    AEG
1755879  0.003992    ADI
1755149  0.004661    IGR
1755894  0.004977   ANSS
1756125  0.005744    SOL
1755144  0.006487   IBOC
1756173  0.007598    WEX
1755794  0.008503    RHI
1756453  0.009266   GOLF
1754896  0.009691     BX
1756380  0.009785   CAAS
1756328  0.009818    NWN
1756818  0.010387    UBP
1754489  0.010755  EMITF
1755986  0.010880    FIS
1755701  0.011737  KELYA
1756900  0.013003    MRO
R2
0.003916559172075007
Model accurate within 10%
0.03961322520274485
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
WFC     0.000232  1.568559e+08  2.271274e+09         1.022733  0.997898   
AAPL   -0.000015  5.895943e+08  2.288687e+09         0.427757  0.971024   
BAC     0.000167  4.215548e+08  2.971961e+09         1.022741  0.936866   
DELL   -0.000008  1.042547e+08

In [199]:
date_start = '2009-04-21'
date_end = '2009-07-21'
allocation_model(df, prediction, date_start, date_end, linreg3, '3M')
allocation_model(df, prediction, date_start, date_end, lasso3, '3M')
allocation_model(df, prediction, date_start, date_end, ridge3, '3M')


            error ticker
1796794  0.001986    AEG
1797389  0.003641    ROC
1797625  0.003831    ETN
1796765  0.004210    SQM
1798582  0.007899    TTM
1797705  0.008288    KFY
1798641  0.008866   FARM
1797522  0.008959   ADES
1798911  0.009223    NRT
1798444  0.010249    GIL
1797537  0.011639    AOS
1796710  0.013271    IPI
1797730  0.016964    MMM
1797420  0.017126    SLM
1797902  0.017320    ASX
1798074  0.017512    OLO
1797691  0.017831    ITW
1798028  0.018807   KOPN
1798084  0.019930   PLUS
1798137  0.020815   SWKS
R2
-0.0027310626482761613
Model accurate within 10%
0.03813953488372093
            beta       volume     marketcap  expected_return  accuracy  \
ticker                                                                   
BAC     0.000178  868366519.0  7.606891e+09         0.843799  0.962733   
AAPL    0.000057  470684032.0  2.046817e+09         0.345670  0.986208   
AXP     0.000149   43608746.0  8.534232e+08         0.843766  0.978244   
MSFT    0.000045   60928280.0  1.

In [200]:
date_start = '2009-06-19'
date_end = '2009-07-21'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')


            error ticker
1935303  0.000067    EWW
1933361  0.000598    LNC
1933881  0.002075    KNX
1933753  0.002538   CHRW
1934749  0.002689    THO
1934011  0.002885    TRI
1934711  0.003816   PRTS
1935507  0.003899  METpA
1935664  0.004675    SFY
1934600  0.005285   ESCA
1934089  0.006815    BHE
1935226  0.007920    DGS
1935403  0.009385    IDV
1935151  0.009733    AMJ
1933870  0.010306    JBT
1935555  0.011247    PGJ
1933481  0.013316    OFG
1933507  0.014328    PFG
1933928  0.014602    NVX
1933355  0.015717    LAZ
R2
-0.0014340605700762232
Model accurate within 10%
0.04240173320953265
            beta      volume     marketcap  expected_return  accuracy  \
ticker                                                                  
FITB    0.000361  68096857.0  4.998309e+08         0.343804  0.989601   
ORCL   -0.000076  51092929.0  1.055580e+09         0.077857  0.999632   
LNC     0.000422  15826854.0  2.668408e+08         0.343804  1.000000   
HIG     0.000381  18142523.0  2.240602

In [201]:
date_start = '2009-02-02'
date_end = '2009-03-02'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')


            error ticker
1622711  0.000130   EZPW
1623408  0.000655    MEG
1623465  0.000679   PLUG
1622695  0.000879    ETG
1623477  0.001822   RCMT
1622956  0.003013    NFJ
1622492  0.004579    AIG
1623333  0.004880     GE
1623474  0.005851   RAIL
1623974  0.007343    BEP
1623703  0.007540    IOT
1622423  0.007551    NEM
1622487  0.007672    AFT
1622895  0.007672    MGI
1622996  0.009132    OPY
1623496  0.009615    SNA
1624028  0.010419    SRE
1624646  0.011901    WES
1623613  0.012048   CGNX
1623070  0.012076    RGA
R2
0.002827006966995893
Model accurate within 10%
0.0374414976599064
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
WFC    -0.000681  8.199108e+07  1.576688e+09        -0.220517  0.994803   
ORCL   -0.000316  2.546632e+07  4.303808e+08        -0.085090  0.998293   
GE     -0.000440  1.690196e+07  1.634055e+09        -0.140095  0.999976   
CSX    -0.000417  5.023216e+07 

In [202]:
date_start = '2009-03-02'
date_end = '2009-04-01'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')

            error ticker
1683596  0.000403    MLM
1683808  0.000913    COF
1684852  0.001088   EMKR
1684641  0.001481   OPTT
1684461  0.004441    CNI
1684695  0.006116    SSD
1685989  0.006361   ENTR
1684885  0.007009   HLIT
1684378  0.007739    WFC
1685575  0.007883    SAM
1684655  0.008072   PKOH
1683660  0.008285     AB
1684517  0.008835   FSTR
1683408  0.009475   LIFE
1684728  0.009811    UNP
1684543  0.010637    HEI
1684026  0.011230    JEF
1685686  0.011328     RC
1684480  0.012734    DCI
1683325  0.015078   CRIS
R2
0.0032978015911391267
Model accurate within 10%
0.04286154794942954
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
WFC     0.000136  1.616336e+08  1.752108e+09         0.690729  0.999940   
AAPL    0.000051  7.709719e+08  2.421391e+09         0.270892  0.953497   
CSCO    0.000028  5.992453e+07  8.581193e+08         0.270891  0.994441   
DELL    0.000025  1.399550e+0

In [203]:
date_start = '2009-04-01'
date_end = '2009-05-01'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')

            error ticker
1755109  0.000841   HARL
1755950  0.000905    CYN
1754555  0.001140     MD
1754854  0.001449   BERK
1756014  0.003540   INFA
1754565  0.004197    MTD
1754472  0.006113    CYT
1755664  0.006142    HEI
1755943  0.006400   CSIQ
1755849  0.006618    UNF
1756963  0.007146   VIST
1755635  0.007273    FLS
1754521  0.008218   ICAD
1755836  0.008763    TRC
1754935  0.009177    CLM
1756161  0.009341    UMC
1756076  0.009847    OLO
1755172  0.012113     KB
1755233  0.012664    MPV
1755391  0.012873    ROC
R2
0.003690984483937232
Model accurate within 10%
0.03462258265751716
            beta        volume     marketcap  expected_return  accuracy  \
ticker                                                                    
AAPL   -0.000015  5.895943e+08  2.288687e+09         0.139370  0.996124   
C       0.000059  3.856367e+07  1.033506e+09         0.335334  0.998912   
DELL   -0.000008  1.042547e+08  2.786520e+08         0.139370  0.996509   
JPM     0.000036  1.060299e+08

In [204]:
date_start = '2009-05-01'
date_end = '2009-06-01'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')

            error ticker
1824304  0.000852   KEQU
1823887  0.006336   NATI
1822326  0.006499    GSK
1823502  0.006539   HSII
1823100  0.007308   NKSH
1823190  0.007783    PTY
1823562  0.009145   MRTN
1822945  0.012316    HPF
1823813  0.013015   FLEX
1824645  0.014741    VNO
1822687  0.017055    BLK
1822328  0.017661   HALO
1823070  0.019305    MXF
1822677  0.019555    BHB
1822969  0.019856    IGD
1822342  0.020521   IDRA
1823114  0.022807    NSL
1823416  0.023619   CPLP
1822802  0.023668   EBTC
1823414  0.026772     CP
R2
0.00026379802382170414
Model accurate within 10%
0.027312228429546864
            beta       volume     marketcap  expected_return  accuracy  \
ticker                                                                   
AAPL   -0.000126  397588940.0  1.806763e+09         0.058689  0.975575   
C      -0.000044   25343834.0  7.527119e+08         0.145411  0.993652   
GS     -0.000056   17526442.0  2.227260e+09         0.145411  0.974399   
DOW    -0.000135   18126768.0  2

In [205]:
date_start = '2009-06-01'
date_end = '2009-07-01'
allocation_model(df, prediction, date_start, date_end, linreg, '1M')
allocation_model(df, prediction, date_start, date_end, lasso, '1M')
allocation_model(df, prediction, date_start, date_end, ridge, '1M')

            error ticker
1890085  0.000349    PEJ
1889633  0.002347    OXY
1889624  0.003544     NR
1888450  0.003697    NOC
1888582  0.003922    ADI
1888520  0.003960    SSD
1889736  0.006078  CMSpB
1889435  0.006945    ARL
1890184  0.007400    RYF
1888275  0.007755   CHMP
1888536  0.008178    TKR
1889827  0.008945    EWA
1890091  0.009162    PFM
1889408  0.009597    TAP
1890301  0.009738     VV
1889353  0.009841   FIZZ
1888374  0.009932    HSC
1888302  0.010832    DAL
1888342  0.011806   FWRD
1889977  0.013036    IVV
R2
0.004552549703897402
Model accurate within 10%
0.04386503067484662
            beta       volume     marketcap  expected_return  accuracy  \
ticker                                                                   
WFC    -0.001520  122382453.0  3.112186e+09        -0.038511  0.997580   
JPM    -0.001495   90193707.0  3.256895e+09        -0.038511  0.999811   
CSCO   -0.001503  103321405.0  2.014767e+09        -0.033826  0.979448   
MRK    -0.001065   20244656.0  5.57